# One Month Death

In [1]:
import pickle
file = open("pickle/1MD/1MD_imputation.pkl",'rb')
dataset = pickle.load(file)

In [2]:
# ----------  init features selection----------
from ProjectML.monthDeath.pre_processing import *
from ProjectML.monthDeath.feature_extraction import *
X, y, dataset = extract_feature(dataset)
X = drop_corr_feature(X, 0.6)
X = best_eight_features(X)
# ----------  end features selection----------


In [3]:
# load model
file = open("pickle/1MD/model_xgb.pkl",'rb')
xgb = pickle.load(file)

C:\Users\utente\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.22.1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [4]:
# evaluation
from ProjectML.general_util.evaluation import *
print("Cross validation cv = 5 ")
balanced_accuracy_score = get_balanced_accuracy(X, y, xgb)
print("Balanced accuracy: %0.2f (+/- %0.2f)" % (balanced_accuracy_score.mean(), balanced_accuracy_score.std() * 2))
f1_score = get_f1_scores(X, y, xgb)
print("f1_score: %0.2f (+/- %0.2f)" % (f1_score.mean(), f1_score.std() * 2))

Cross validation cv = 5 
Balanced accuracy: 0.70 (+/- 0.07)
f1_score: 0.50 (+/- 0.10)


# Procedural Success

In [5]:
file2 = open("pickle/PS/PS_imputation.pkl",'rb')
dataset = pickle.load(file2)

In [6]:
# ----------  load datasets ----------
from ProjectML.proceduralSuccess.pre_processing import *
from ProjectML.proceduralSuccess.feature_extraction import *
X, y, dataset = extract_feature(dataset)
mask, gb_coefs, gb_mask, rf_coefs, rf_mask = voting_feature_selection(X, y)
X = X.loc[:,mask]

X_scaled = pd.read_pickle('pickle/PS/PS_scaled_features.pkl')
X_scaled = X_scaled.loc[:, ['CenterID','PatientID']]


Fitting estimator with 70 features.
Fitting estimator with 65 features.
Fitting estimator with 60 features.
Fitting estimator with 55 features.
Fitting estimator with 50 features.
Fitting estimator with 45 features.
Fitting estimator with 40 features.
Fitting estimator with 35 features.
Fitting estimator with 70 features.
Fitting estimator with 65 features.
Fitting estimator with 60 features.
Fitting estimator with 55 features.
Fitting estimator with 50 features.
Fitting estimator with 45 features.
Fitting estimator with 40 features.
Fitting estimator with 35 features.


## Random Forest

In [7]:
# load model
file2 = open("pickle/PS/model_random_forest.pkl",'rb')
rf= pickle.load(file2)

C:\Users\utente\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\utente\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
# ---------- init split test ----------
X_train_val, X_test, y_train_val, y_test = my_l_split(X, y, 0.1)
X_train_val_scal, X_test_scal, y_train_val_scal, y_test_scal = my_l_split(X_scaled, y, 0.1)
# ---------- end split test ----------


# ---------- init split train validation ----------
X_train, X_val, y_train, y_val = my_l_split(X_train_val, y_train_val, 2 / 9)
X_train_scal, X_val_scal, y_train_scal, y_val_scal = my_l_split(X_train_val_scal, y_train_val_scal, 2 / 9)
# ---------- end split train validation ----------


In [9]:
#evaluation
from ProjectML.general_util.evaluation import *
print("Classification report on test set:")
y_pred = rf.predict(X_test)
rep_test = classification_report(y_test, y_pred)
print(rep_test)

Classification report on test set:
              precision    recall  f1-score   support

       False       1.00      0.95      0.97        41
        True       0.99      1.00      1.00       299

    accuracy                           0.99       340
   macro avg       1.00      0.98      0.99       340
weighted avg       0.99      0.99      0.99       340



In [10]:
#cross validation f1 score
clf = RandomForestClassifier(n_estimators=500, criterion='gini', max_features=5, n_jobs=-1, \
                             class_weight='balanced',random_state=SEED)
print("Cross validation cv = 10 ")
f1_mean0, f1_std0, f1_mean1, f1_std1 = my_cross_f1(X, y, clf, cv=10)
print("f1_score: %0.2f (+/- %0.2f) (class 0)" % (f1_mean0, f1_std0 * 1.95))
print("f1_score: %0.2f (+/- %0.2f) (class 1)" % (f1_mean1, f1_std1 * 1.95))

Cross validation cv = 10 
f1_score: 0.94 (+/- 0.03) (class 0)
f1_score: 0.99 (+/- 0.00) (class 1)


## SVM

In [11]:
# load model
file2 = open("pickle/PS/model_svm.pkl",'rb')
model = pickle.load(file2)

C:\Users\utente\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.22.1 when using version 0.22. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [12]:
from ProjectML.general_util.evaluation import *
print("Classification report on test set:")
y_pred = model.predict(X_test_scal)
rep_test = classification_report(y_test_scal, y_pred)
print(rep_test)

Classification report on test set:
              precision    recall  f1-score   support

       False       0.87      0.95      0.91        41
        True       0.99      0.98      0.99       299

    accuracy                           0.98       340
   macro avg       0.93      0.97      0.95       340
weighted avg       0.98      0.98      0.98       340



In [13]:
#cross validation f1 score
from sklearn import svm
clf = svm.SVC(C=100.0, kernel='rbf', class_weight='balanced', max_iter=-1, random_state=SEED)
print("SVM: cross validation cv = 10 ")
f1_mean0, f1_std0, f1_mean1, f1_std1 = my_cross_f1(X_scaled, y, clf, cv=10)
print("f1_score: %0.2f (+/- %0.2f) (class 0)" % (f1_mean0, f1_std0 * 1.95))
print("f1_score: %0.2f (+/- %0.2f) (class 1)" % (f1_mean1, f1_std1 * 1.95))

SVM: cross validation cv = 10 
f1_score: 0.88 (+/- 0.07) (class 0)
f1_score: 0.98 (+/- 0.01) (class 1)
